In [1]:
from agential.cog.agent.critic import CriticAgent
from agential.cog.agent.critic_old import CriticAgent as OldCriticAgent
from langchain_community.utilities.google_search import GoogleSearchAPIWrapper

from langchain_community.chat_models.openai import ChatOpenAI
from agential.cog.prompts.critic import (
    CRITIC_INSTRUCTION_HOTPOTQA,
    HOTPOTQA_FEWSHOT_EXAMPLES_COT,
    HOTPOTQA_FEWSHOT_EXAMPLES_DIRECT,
    HOTPOTQA_FEWSHOT_EXAMPLES_REACT,
    CRITIC_CRITIQUE_INSTRUCTION_HOTPOTQA,
    HOTPOTQA_FEWSHOT_EXAMPLES_CRITIC,

    CRITIC_INSTRUCTION_FEVER,
    FEVER_FEWSHOT_EXAMPLES_COT,
    FEVER_FEWSHOT_EXAMPLES_DIRECT,
    FEVER_FEWSHOT_EXAMPLES_REACT,
    CRITIC_CRITIQUE_INSTRUCTION_FEVER,
    FEVER_FEWSHOT_EXAMPLES_CRITIC,

    CRITIC_INSTRUCTION_AMBIGNQ,
    AMBIGNQ_FEWSHOT_EXAMPLES_COT,
    AMBIGNQ_FEWSHOT_EXAMPLES_DIRECT,
    AMBIGNQ_FEWSHOT_EXAMPLES_REACT,
    CRITIC_CRITIQUE_INSTRUCTION_AMBIGNQ,
    AMBIGNQ_FEWSHOT_EXAMPLES_CRITIC,

    CRITIC_INSTRUCTION_TRIVIAQA,
    TRIVIAQA_FEWSHOT_EXAMPLES_COT,
    TRIVIAQA_FEWSHOT_EXAMPLES_DIRECT,
    TRIVIAQA_FEWSHOT_EXAMPLES_REACT,
    CRITIC_CRITIQUE_INSTRUCTION_TRIVIAQA,
    TRIVIAQA_FEWSHOT_EXAMPLES_CRITIC,
)

import dotenv
dotenv.load_dotenv()

import warnings
warnings.filterwarnings("ignore")

llm = ChatOpenAI()
search = GoogleSearchAPIWrapper()

In [2]:
agent = CriticAgent(
    llm=llm, mode="qa", search=search
)

In [3]:
use_search_tool = False

In [4]:
# HotpotQA
question = 'Who was once considered the best kick boxer in the world, however he has been involved in a number of controversies relating to his "unsportsmanlike conducts" in the sport and crimes of violence outside of the ring'
out = agent.generate(
    question=question,
    examples=HOTPOTQA_FEWSHOT_EXAMPLES_COT,
    prompt=CRITIC_INSTRUCTION_HOTPOTQA,
    critique_examples=HOTPOTQA_FEWSHOT_EXAMPLES_CRITIC,
    critique_prompt=CRITIC_CRITIQUE_INSTRUCTION_HOTPOTQA,
    use_search_tool=use_search_tool
)

<PROMPT AGENT===========================================================>
Q: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
A: Let's think step by step. The eastern sector of Colorado orogeny extends into the High Plains. High Plains rise in elevation from around 1,800 to 7,000 ft. So the answer is: 1,800 to 7,000 ft.

Q: Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?
A: Let's think step by step. Milhouse was named after U.S. president Richard Nixon. So the answer is: Richard Nixon.

Q: Which documentary is about Finnish rock groups, Adam Clayton Powell or The Saimaa Gesture?
A: Let's think step by step. Adam Clayton Powell (film) is a documentary about an African-American politician, not Finnish rock groups. So the documentary about Finnish rock groups must instead be The Saimaa Gesture. So the answer is: The Saimaa Gesture.

Q: What profession does 

In [5]:
out

[{'answer': "Let's break this down step by step. The clues indicate that this person was once considered the best kickboxer in the world, but has been involved in controversies related to unsportsmanlike conduct and crimes of violence. One famous kickboxer who fits this description is Badr Hari. So the answer is: Badr Hari.",
  'critique': '\n\nThe answer seems plausible as it provides a name that fits the description given in the question.\n\n2. Truthfulness:\n\nLet\'s search the question in Google:\n\n> Search Query: Who was once considered the best kick boxer in the world, however he has been involved in a number of controversies relating to his "unsportsmanlike conducts" in the sport and crimes of violence outside of the ring\n> Evidence: The search results do not provide specific information about a kickboxer who fits the description given in the question.\n\nLet\'s search the proposed answer in Google:\n\n> Search Query: Badr Hari controversies and unsportsmanlike conduct',
  'se

In [6]:
# FEVER
question = "Nikolaj Coster-Waldau worked with the Fox Broadcasting Company."
out = agent.generate(
    question=question,
    examples=FEVER_FEWSHOT_EXAMPLES_COT,
    prompt=CRITIC_INSTRUCTION_FEVER,
    critique_examples=FEVER_FEWSHOT_EXAMPLES_CRITIC,
    critique_prompt=CRITIC_CRITIQUE_INSTRUCTION_FEVER,
    use_search_tool=use_search_tool
)

<PROMPT AGENT===========================================================>
Claim: Did Nikolaj Coster-Waldau work with the Fox Broadcasting Company?
A: Yes, he appeared in the 2009 Fox television film Virtuality. So the answer is: SUPPORTS.

Claim: Is Stranger Things set in Bloomington, Indiana?
A: No, it is set in the fictional town of Hawkins, Indiana. So the answer is: REFUTES.

Claim: Did the song "Beautiful" by Christina Aguilera reach number two on the Billboard Hot 100 in 2003?
A: The song peaked at number two on the Billboard Hot 100, but it does not specify that it was in 2003. So the answer is: NOT ENOUGH INFO.
(END OF EXAMPLES)

Claim: Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.
A: 
<PROMPT AGENT===========================================================>
<OUT AGENT===========================================================>
'SUPPORTS. Nikolaj Coster-Waldau appeared in the 2009 Fox television film Virtuality.'
<OUT AGENT=====================================

In [8]:
out

[{'answer': 'SUPPORTS. Nikolaj Coster-Waldau appeared in the 2009 Fox television film Virtuality, confirming his work with the Fox Broadcasting Company.',
  'critique': 'SUPPORTS. Nikolaj Coster-Waldau appeared in the 2009 Fox television film Virtuality, confirming his work with the Fox Broadcasting Company.'}]

In [9]:
# AmbigNQ
question = "When did the simpsons first air on television?"
out = agent.generate(
    question=question,
    examples=AMBIGNQ_FEWSHOT_EXAMPLES_COT,
    prompt=CRITIC_INSTRUCTION_AMBIGNQ,
    critique_examples=AMBIGNQ_FEWSHOT_EXAMPLES_CRITIC,
    critique_prompt=CRITIC_CRITIQUE_INSTRUCTION_AMBIGNQ,
    use_search_tool=use_search_tool
)

<PROMPT AGENT===========================================================>
Q: What airport is closest to Palm Springs?
A: The nearest airport to Palm Springs is Indio/Palm Springs (PSP) Airport which is 2.1 miles away. So the answer is: Palm Springs International Airport

Q: What degree did Martin Luther King get?
A: Martin Luther King earned his Bachelor of Divinity degree from Crozer Theological Seminary, followed by a doctorate in Systematic Theology from Boston University. So the answer is: Bachelor of Divinity

Q: What countries does the Niger river flow through?
A: The Niger river runs in a crescent through Libya, Mali, Niger, on the border with Benin and then through Nigeria. So the answer is: Libya

Q: What type of currency is used in Puerto Rico?
A: Puerto Rico is a territory of the United States and uses the U.S. dollar. So the answer is: United States dollar

Q: Who played kitt in knight rider?
A: kitt was voice most often by William Daniels. So the answer is: William Daniels

In [10]:
out

[{'answer': 'The Simpsons first aired on television on December 17, 1989.',
  'critique': '\n\nThe question asks for a time, and the answer provides a specific date of December 17, 1989. So it\'s plausible. \n\n2. Truthfulness:\n\nLet\'s search the question in google:\n\n> Search Query: When did The Simpsons first air on television?\n> Evidence: [The Simpsons - Wikipedia] The Simpsons is an American animated sitcom created by Matt Groening for the Fox Broadcasting Company. The series first premiered on December 17, 1989.\n\nThe evidence confirms that The Simpsons first aired on television on December 17, 1989, validating the proposed answer.\n\nTherefore, the proposed answer "The Simpsons first aired on television on December 17, 1989" is accurate and truthful.',
  'search_query': 'When did The Simpsons first air on television?',
  'search_result': '[The Simpsons - Wikipedia] The Simpsons is an American animated sitcom created by Matt Groening for the Fox Broadcasting Company. The seri

In [11]:
# TriviaQA
question = "Which American-born Sinclair won the Nobel Prize for Literature in 1930?"
out = agent.generate(
    question=question,
    examples=TRIVIAQA_FEWSHOT_EXAMPLES_COT,
    prompt=CRITIC_INSTRUCTION_TRIVIAQA,
    critique_examples=TRIVIAQA_FEWSHOT_EXAMPLES_CRITIC,
    critique_prompt=CRITIC_CRITIQUE_INSTRUCTION_TRIVIAQA,
    use_search_tool=use_search_tool
)

<PROMPT AGENT===========================================================>
Q: Mendelssohn's 'Wedding March' was. originally written as incidental music for which Shakespeare play in 1842? 
A: Mendelssohn's 'Wedding March' was originally written as incidental music for A Midsummer Night's Dream in 1842. So the answer is: A Midsummer Night's Dream.

Q: Christ in the House of his Parents"" is one of the best known paintings of which artist?" 
A: "Christ in the House of his Parents" is a painting by John Everett Millais. So the answer is: John Everett Millais.

Q: Who designed the National Theatre building on the South Bank in London ? 
A: The National Theatre building on the South Bank in London was designed by Denys Lasdun. So the answer is: Denys Lasdun.

Q: Also a two-time World Champion, which American skier won the gold medal in the Men's Combined at the 2010 Winter Olympics? 
A: The only American skier who won the gold medal in the Men's Combined at the 2010 Winter Olympics is Bode M

In [12]:
out

[{'answer': 'Upton Sinclair, an American author, won the Nobel Prize for Literature in 1930.',
  'critique': '\n\nThe question asks for the name of the American-born Sinclair who won the Nobel Prize for Literature in 1930, and the answer is "Upton Sinclair", which is a plausible name. So it\'s plausible.\n\n2. Truthfulness:\n\nLet\'s search the question in google:\n\n> Search Query: Which American-born Sinclair won the Nobel Prize for Literature in 1930? site: wikipedia.org \n> Evidence: [Winners of the Nobel Prize in Literature - Wikipedia] The Nobel Prize in Literature for 1930 was awarded to Sinclair Lewis "for his vigorous and graphic art of description and his ability to create, with wit and humor, new types of characters."\n\nThe evidence shows that the American-born Sinclair who won the Nobel Prize for Literature in 1930 was Sinclair Lewis, not Upton Sinclair.\n\nIn summary, the proposed answer "Upton Sinclair" is incorrect, and the correct answer is Sinclair Lewis. \n\nQuestion